In [19]:
import geopandas as gpd
import pandas as pd
import numpy as np

import geoengine as ge



In [31]:
ge.initialize("http://localhost:3030/api")#, credentials=("admin@localhost", "admin"))

In [32]:
points_df = gpd.read_file("group_sample_frac1_inspireId.gpkg")
points_id = ge.upload_dataframe(points_df, "group_sample_frac1_inspireId")
points_id

6ae36c4e-537c-45e5-b1a2-9179f384202a

In [22]:
points_source_operator = ge.unstable.workflow_operators.OgrSource(str(points_id))
points_source_operator.to_workflow_dict()

{'type': 'Vector',
 'operator': {'type': 'OgrSource',
  'params': {'data': {'type': 'internal',
    'datasetId': '2c9cda87-3ea3-4f38-99ad-b2725b22baf2'},
   'attributeProjection': None,
   'attributeFilters': None}}}

In [23]:
bounds_array = [421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239]
xmin = bounds_array[0]
ymin = bounds_array[1]
xmax = bounds_array[2]
ymax = bounds_array[3]

(xmin, ymin, xmax, ymax)

(421398.30688394164, 5681079.293017321, 476200.5934965983, 5727832.747780239)

In [24]:
from datetime import datetime
time_start = datetime(2021, 1, 1)
time_end = datetime(2021, 12, 31)

study_area = ge.api.RasterQueryRectangle(
    spatialBounds=ge.SpatialPartition2D(xmin, ymin, xmax, ymax).to_api_dict(),
    timeInterval=ge.TimeInterval(time_start, time_end).to_api_dict(),
    spatialResolution=ge.SpatialResolution(10.0, 10.0).to_api_dict(),
)
study_area

{'spatialBounds': {'upperLeftCoordinate': {'x': 421398.30688394164,
   'y': 5727832.747780239},
  'lowerRightCoordinate': {'x': 476200.5934965983, 'y': 5681079.293017321}},
 'timeInterval': {'start': '2021-01-01T00:00:00.000+00:00',
  'end': '2021-12-31T00:00:00.000+00:00'},
 'spatialResolution': {'x': 10.0, 'y': 10.0}}

In [34]:
# TASK workflows to datasets ~ 200 min



band_data_map = {
    'b02': '48d30acd-a378-4f2a-89b3-d73430c0f29e',
    'b03': '200ed06c-fcd1-41a3-b2dd-68b5d0b338a9',
    'b04': 'b9ececef-5a0c-4294-be16-334e8017f60f',
    'b08': '8e635367-35ff-42f9-bb21-88ae9db8be8d',
    'scl': '64636605-10da-4576-bf21-a265cc1f7d9c'
    }

band_data_id_map = {name: 
    ge.api.InternalDataId(
        type="internal",
        datasetId=id) for name, id in band_data_map.items()
} 

band_data_id_map


{'b02': {'type': 'internal',
  'datasetId': '48d30acd-a378-4f2a-89b3-d73430c0f29e'},
 'b03': {'type': 'internal',
  'datasetId': '200ed06c-fcd1-41a3-b2dd-68b5d0b338a9'},
 'b04': {'type': 'internal',
  'datasetId': 'b9ececef-5a0c-4294-be16-334e8017f60f'},
 'b08': {'type': 'internal',
  'datasetId': '8e635367-35ff-42f9-bb21-88ae9db8be8d'},
 'scl': {'type': 'internal',
  'datasetId': '64636605-10da-4576-bf21-a265cc1f7d9c'}}

In [35]:
sentinel_bands = ["B02", "B03", "B04", "B08", "NDVI"]
operators = {b: ge.unstable.workflow_blueprints.s2_cloud_free_monthly_band(b, band_data_id_map, granularity="months", window_size=1, aggregation_type="mean") for b in sentinel_bands}

projected_points = ge.unstable.workflow_operators.Reprojection(points_source_operator, target_spatial_reference="EPSG:32632")

points_with_s2_cloud_free = ge.unstable.workflow_operators.RasterVectorJoin(
    raster_sources=[x for x in operators.values() ],
    vector_source=projected_points,
    new_column_names=[x for x in operators.keys()],
)

points_with_s2_cloud_free.to_dict()
        

AttributeError: module 'geoengine.unstable.workflow_blueprints' has no attribute 's2_cloud_free_monthly_band'

In [27]:
workflow = ge.register_workflow(points_with_s2_cloud_free.to_workflow_dict())
workflow

31c4f598-e137-50a7-8859-ab9eb166d8e9

In [33]:
import datetime

start_dt = datetime.datetime(2021, 4, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
end_dt = datetime.datetime(2021, 10, 30, 0, 0, 0, tzinfo=datetime.timezone.utc)


async for x in workflow.vector_stream(
    ge.QueryRectangle(
        spatial_bounds=ge.BoundingBox2D(
            xmin=xmin,
            ymin=ymin,
            xmax=xmax,
            ymax=ymax,
        ),
        time_interval=ge.TimeInterval(
            start=start_dt,
            end=end_dt,
        ),
        resolution=ge.SpatialResolution(
            100.0,
            100.0,
        ),
        srs="EPSG:32632",
)):
    print (x)



InvalidStatusCode: server rejected WebSocket connection: HTTP 400